<a href="https://colab.research.google.com/github/Damilola-ops/Text-de-shuffling/blob/main/Text_de_shuffling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.2.1
!pip install datasets==1.0.2
!pip install torch==1.6.0
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 37.3 MB/s 
     |████████████████████████████████| 880 kB 51.2 MB/s 
     |████████████████████████████████| 2.9 MB 52.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9ba3c2ee24cc4a94317f71c6fc60e149b9c11e926d98d5b3bc293e54f40ba65e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 30.0 MB/s 
     |████████████████████████████████| 212 kB 11.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installat

In [ ]:
import wandb
from transformers import GPT2LMHeadModel,GPT2Tokenizer
from transformers import EncoderDecoderModel,EncoderDecoderConfig,RobertaTokenizerFast
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
from transformers import BertTokenizerFast
import torch
import pandas as pd 
import numpy as np
import sklearn.preprocessing 
from torch.nn import functional as F 
import datasets
from datasets import load_dataset
import psutil

In [ ]:
wandb.init(project='Text-Descrambling-BERT2BERT',
           entity='damilojohn',
           name = 'No Decoding Strategies'
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damilojohn. Use `wandb login --relogin` to force relogin


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv'
test_path = '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv'
val_path= '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv'

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
val_df = pd.read_csv(val_path)

In [ ]:
#using huggingface's dataset function to load multiple datasets at once
val_dataset = datasets.Dataset.from_pandas(val_df) 
dataset = load_dataset( 'csv',data_files = {'train':[train_path],
                                         'test':[test_path]}
                       )

Downloading:   0%|          | 0.00/887 [00:00<?, ?B/s]

Using custom data configuration default


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4ba4dd19b06fc75e/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277. Subsequent calls will reuse this data.


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#testing out our tokenizer
sample = dataset['train']['text'][10]
tokens = tokenizer.tokenize(sample)
token_id = tokenizer.convert_tokens_to_ids(tokens)
original_text = tokenizer.decode(token_id)
print('sample text: ', sample)
print('tokens : ',tokens)
print('token_id : ',token_id)
print(original_text)

sample text:  Properties and packing. and tertiary were primary, secondary used structures, accessibility
tokens :  ['properties', 'and', 'packing', '.', 'and', 'tertiary', 'were', 'primary', ',', 'secondary', 'used', 'structures', ',', 'accessibility']
token_id :  [5144, 1998, 14743, 1012, 1998, 13553, 2020, 3078, 1010, 3905, 2109, 5090, 1010, 23661]
properties and packing. and tertiary were primary, secondary used structures, accessibility


In [ ]:
#a function to tokenize all the rows in our dataset
max_text_length = 40 
max_label_length = 40 
def preprocess_function(sample):
    # Getting text and label
    text = sample["text"]
    label = sample["label"]
    # Tokenizing the text and label
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_text_length)
    outputs = tokenizer(label, padding="max_length", truncation=True, max_length=max_label_length,)
    sample["input_ids"] = inputs.input_ids
    sample["attention_mask"] = inputs.attention_mask
    sample["decoder_input_ids"] = outputs.input_ids
    sample["decoder_attention_mask"] = outputs.attention_mask
    sample["labels"] = outputs.input_ids
    # The labels are used to calcuate the loss while training, and because we added padding to make all tokens to be of same size,
    # we also need to convert the padding number ( 0 ) to ( -100 ), so that we can tell huggingface that these number can be ignorned while calcuating loss. 
    # Why specifically -100 ? It's simply an arbitrary number, again so that huggingface can ignore this number while calcuating loss
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]

    return sample

In [ ]:
#metrics 
def WER():
  pass


In [ ]:
batch = 16
tokenized_dataset=dataset.map(preprocess_function,batch_size=batch,batched=True)
val_tokenized = val_dataset.map(preprocess_function,batch_size=batch,batched=True)
tokenized_dataset.remove_columns_(['text','label'])
val_tokenized.remove_columns_(['text','label'])

  0%|          | 0/2501 [00:00<?, ?ba/s]

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/251 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask",'decoder_input_ids',"decoder_attention_mask", "labels"],
)
val_tokenized.set_format(
    type='torch',columns=['input_ids','attention_mask','decoder_input_ids','decoder_attention_mask','labels']
)

In [ ]:
Model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased','bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer

In [ ]:
Model.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "eos_token_id": null,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "label2id": {
      "LABEL_0": 0,
      "LABEL_1": 1
    },
    "layer_norm_eps": 1e-12,
    "length_penalty": 1.0,
    "max_length": 20,
    "max_position_embeddings": 512,
    "min_length": 0,
    "model_type": "bert",


In [ ]:
Model.config.decoder_start_token_id = tokenizer.cls_token_id
Model.config.eos_token_id = tokenizer.sep_token_id
Model.config.pad_token_id = tokenizer.pad_token_id
Model.config.vocab_size = tokenizer.vocab_size
Model.config.max_length = 142
Model.config.min_length = 56





In [ ]:
N_epochs = 10
BATCH_SIZE=32
args = Seq2SeqTrainingArguments(
    "Text De-Scrambling",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=N_epochs,
    predict_with_generate=True,
    save_steps=2000,
    fp16=True,
    # This will help in increasing the speed of training
)

In [ ]:
trainer = Seq2SeqTrainer(model = Model,
                         args = args,
                         train_dataset = tokenized_dataset['train'],
                         eval_dataset = val_tokenized,
                         tokenizer=tokenizer
                         )

                                            
                         
                         

In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.htm

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,1.948100,1.154155,11.453000,349.341000
2,0.913200,0.805200,11.480300,348.509000
3,0.648800,0.664261,11.468200,348.879000
4,0.458700,0.618649,11.739500,340.816000


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,1.948100,1.154155,11.453000,349.341000
2,0.913200,0.805200,11.480300,348.509000
3,0.648800,0.664261,11.468200,348.879000
4,0.458700,0.618649,11.739500,340.816000
5,0.336300,0.632333,11.470900,348.796000
6,0.244600,0.601728,11.438300,349.790000
7,0.180000,0.606807,11.390100,351.269000
8,0.131500,0.598431,12.998100,307.814000
9,0.099000,0.610831,11.473700,348.710000
10,0.076800,0.607422,11.494500,348.081000


TrainOutput(global_step=12510, training_loss=0.5717279166912289, metrics={'train_runtime': 5286.6138, 'train_samples_per_second': 2.366, 'total_flos': 47494957456252800, 'epoch': 10.0})

In [20]:
def generate_predictions(batch):

    # Tokenizing the test
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_text_length, return_tensors="pt")
    
    # Sending the tensors to GPU
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # Generating the predicted tokens ids
    outputs = Model.generate(input_ids, attention_mask=attention_mask)

    # Converting the token ids to sentence
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["predictions"] = output_str

    return batch
    
    
    

In [21]:
results = dataset['test'].map(generate_predictions,batched=True,batch_size=16)
test_df['label'] = results['predictions']
test_df

  0%|          | 0/625 [00:00<?, ?ba/s]

,id,text,label
0,0,safely objects move. that system images detect...,"using this approach, skip images can barely ba..."
1,1,We detectors popular influence of confidences ...,we studied the influence of three popular conf...
2,2,compact coding present supervised We a approac...,"we present a compact coding approach, supervis..."
3,3,study high-throughput vital of quantitative be...,high - throughput measurement of collective ve...
4,4,on data sets. We evaluate method many challeng...,we evaluate the proposed method on many challe...
...,...,...,...
9995,9995,"particular i.e. problem, of to due However, na...","however, due to the particular nature of the p..."
9996,9996,Simulation methods. proposed outperforming met...,simulation results demonstrate that the propos...
9997,9997,in This view introduces a scenarios. paper tec...,this paper introduces a technique for label la...
9998,9998,valve. water from water are and pipeline sourc...,"the pipeline of water pump, motor, interferenc..."


In [31]:
test_df.to_csv('/content/drive/MyDrive/Colab datasets/descrambled_texts.csv')

In [45]:
#testing out the model on a sample instance 
Model.eval()
test_text = dataset['train']['text'][14]
min_length_gen = len(test_text.split(' '))
max_length_gen = len(test_text) 
label = dataset['train']['label'][14]
input = tokenizer(test_text,return_tensors= 'pt',padding='max_length',max_length=max_text_length,truncation=True,)
input = input.to('cuda')
input_id = input.input_ids
attention_mask = input.attention_mask
output = Model.generate(**input,)
output = tokenizer.decode(output[0],skip_special_tokens=True)
output

'first, we used the mammoth - mult structural aligner to align the training set proteins.. the training... the training set used... we used the mammoth - mult training set to align the proteins..........'

In [46]:
label

'First, we used the MAMMOTH-mult structural aligner to align the training set proteins.'

In [30]:
trainer.save_model('/content/drive/MyDrive/Models/Bert2BertForDescrambling')

In [24]:
print(test_text,'\n', label,'\n' ,output)

the co-evolution a presented. classifier by enforcing is inspired fitness new First, function, and diversity, 
 First, a new fitness function, inspired by co-evolution and enforcing the classifier diversity, is presented. 
 first, a new fitness function, inspired by co - evolution and enforcing the classifier diversity, is presented.''.''. the fitness function, inspired by enforcing diversity, is presented by enforcing the fitness, co - evolution....... the
the co-evolution a presented. classifier by enforcing is inspired fitness new First, function, and diversity, 
 First, a new fitness function, inspired by co-evolution and enforcing the classifier diversity, is presented. 
 first, a new fitness function, inspired by co - evolution and enforcing the classifier diversity, is presented.''.''. the fitness function, inspired by enforcing diversity, is presented by enforcing the fitness, co - evolution....... the


In [ ]:
!pip install huggingface_hub 

In [29]:
from huggingface_hub import notebook_login

In [32]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
